Get conn params

In [3]:
from dotenv import dotenv_values
import os
env = dotenv_values("../../.env")
env

OrderedDict([('ENV_NAME', 'pdfbookmarks'),
             ('BACKEND_HOST', 'localhost'),
             ('BACKEND_PORT', '9113'),
             ('REDIS_PORT', '9114'),
             ('REDIS_HOST', 'localhost'),
             ('VITE_PORT', '5175')])

In [4]:
redis_host = env['REDIS_HOST']
redis_port = env['REDIS_PORT']

Connect

In [1]:
import redis

In [5]:
r = redis.Redis(host=redis_host, port=redis_port, decode_responses=True)
r

Redis<ConnectionPool<Connection<host=localhost,port=9114,db=0>>>

In [29]:
# delete all keys
r.flushall()

True

In [123]:
# list all keys
r.keys()

['7019d2b5-75f8-4573-92a2-e25eb8e83af9',
 '7019d2b5-75f8-4573-92a2-e25eb8e83af9:progress']

In [124]:
r.get(f'{r.keys()[0]}:progress')

'1310'

Get and set status of file

In [27]:
from pydantic import BaseModel
from pydantic import Field

In [31]:
class PDFFileStatus(BaseModel):
    filename: str
    is_processing: int = Field(default=0, ge=0, le=1, description="0 if not processing, 1 if processing.")
    processing: float = Field(default=0.0, ge=0.0, le=1.0, description="Processing progress as a float between 0 and 1.")

In [32]:
# delete all keys
r.flushall()

True

In [33]:
# key = fileid
fileid = '1234567890'
r.hset('1234567890', mapping=dict(PDFFileStatus(filename='test.pdf')))

3

In [34]:
r.hgetall(fileid)

{'filename': 'test.pdf', 'is_processing': '0', 'processing': '0.0'}

In [37]:
PDFFileStatus.model_validate(r.hgetall(fileid))

PDFFileStatus(filename='test.pdf', is_processing=0, processing=0.0)